In [1]:
import os
import os.path as osp
os.chdir('/root/share/CLRNet')
import cv2
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import argparse
import numpy as np
import random
from tqdm import tqdm

from clrnet.utils.config import Config
from clrnet.engine.runner import Runner
from clrnet.datasets import build_dataloader

/root/miniconda3/envs/clrnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def parse_args(input_args=[]):
    parser = argparse.ArgumentParser(description='Train a detector')
    parser.add_argument('config', help='train config file path')
    parser.add_argument('--work_dirs',
                        type=str,
                        default=None,
                        help='work dirs')
    parser.add_argument('--load_from',
                        default=None,
                        help='the checkpoint file to load from')
    parser.add_argument('--resume_from',
            default=None,
            help='the checkpoint file to resume from')
    parser.add_argument('--finetune_from',
            default=None,
            help='the checkpoint file to resume from')
    parser.add_argument('--view', action='store_true', help='whether to view')
    parser.add_argument(
        '--validate',
        action='store_true',
        help='whether to evaluate the checkpoint during training')
    parser.add_argument(
        '--test',
        action='store_true',
        help='whether to test the checkpoint on testing set')
    parser.add_argument('--gpus', nargs='+', type=int, default='0')
    parser.add_argument('--seed', type=int, default=0, help='random seed')
    args = parser.parse_args(input_args)

    return args


target = 'clr_dla34_k_road1' # 'clr_dla34_culane'
checkpoint = 'culane_dla34'
args = parse_args([f'configs/clrnet/{target}.py',
                   '--test',
                   '--load_from', f'{checkpoint}.pth',
                   '--gpus', 0,
                   '--view'])

os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(
    str(gpu) for gpu in args.gpus)

cfg = Config.fromfile(args.config)
cfg.gpus = len(args.gpus)

cfg.load_from = args.load_from
cfg.resume_from = args.resume_from
cfg.finetune_from = args.finetune_from
cfg.view = args.view
cfg.seed = args.seed

cfg.work_dirs = args.work_dirs if args.work_dirs else cfg.work_dirs

cudnn.benchmark = True

runner = Runner(cfg)
runner.val_loader = build_dataloader(runner.cfg.dataset.val,
                                    runner.cfg,
                                    is_train=False)
runner.net = runner.net.eval()

2024-11-20 12:07:05,759 - clrnet.utils.recorder - INFO - Config: 
net = dict(type='Detector', )

backbone = dict(
    type='DLAWrapper',
    dla='dla34',
    pretrained=True,
)

num_points = 72 # the number of points of lane prior
max_lanes = 4
sample_y = range(1440-1, 720, -20) # range(ori_img_h - 1, cut_height, -20)'. https://github.com/Turoad/lanedet/issues/57 

heads = dict(type='CLRHead',
             num_priors=192,
             refine_layers=3,
             fc_hidden_dim=64,
             sample_points=36)

iou_loss_weight = 2.
cls_loss_weight = 2.
xyt_loss_weight = 0.2
seg_loss_weight = 1.0

work_dirs = "work_dirs/clr/dla34_k_road1"

neck = dict(type='FPN',
            in_channels=[128, 256, 512],
            out_channels=64,
            num_outs=3,
            attention=False)

test_parameters = dict(conf_threshold=0.4, nms_thres=50, nms_topk=max_lanes)

epochs = 15
batch_size = 24 

optimizer = dict(type='AdamW', lr=0.6e-3)  # 3e-4 for batchsize 8
total_iter = (88880 // batch_

In [6]:
predictions = []
for i, data in enumerate(tqdm(runner.val_loader, desc=f'Validate')):
    data = runner.to_cuda(data)
    with torch.no_grad():
        output = runner.net(data)
        output = runner.net.module.heads.get_lanes(output)
        predictions.extend(output)
    if runner.cfg.view:
        runner.val_loader.dataset.view(output, data['meta'], only_lane=False)

Validate: 100%|██████████| 36/36 [00:30<00:00,  1.18it/s]


or by test command

In [ ]:
python main.py configs/clrnet/clr_dla34_k_road1.py --test --load_from culane_dla34.pth --gpus 0 --view

Check the result at '/root/share/CLRNet/work_dirs/clr/dla34_k_road1/'